In [1]:
import polars as pl
import nltk

In [2]:
nltk.download("stopwords")
nltk.download("words")


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kimaita/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/kimaita/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [3]:
from nltk.corpus import stopwords, words

stops = set(stopwords.words("english"))
word_list = set(words.words("en")).difference(stops)


In [4]:
df_wikt = pl.scan_csv(
    "enwiktionary-latest-all-titles-in-ns0",
    new_columns=["title"],
    separator="\n",
).filter(pl.col("title").is_in(word_list))


In [5]:
from wordfreq import get_frequency_dict

eng_freq = get_frequency_dict("en", wordlist="best")


In [6]:
df_freq = (
    df_wikt.with_columns(
        freq=pl.col("title").str.to_lowercase().replace_strict(eng_freq, default=1e-09)
    )
    .sort("freq", descending=True)
    .collect()
)


In [7]:
df_freq.describe()


statistic,title,freq
str,str,f64
"""count""","""185263""",185263.0
"""null_count""","""0""",0.0
"""mean""",null,0.000002
"""std""",null,0.000068
"""min""","""A""",1.0000e-9
"""25%""",null,1.0000e-9
"""50%""",null,1.0000e-9
"""75%""",null,4.2658e-8
"""max""","""zythum""",0.022909


In [343]:
df_freq.sample(n=10)

title,freq
str,f64
"""phaseal""",1.0000e-9
"""picker""",9.7724e-7
"""rollmop""",1.0000e-9
"""prevailer""",1.0000e-9
"""countermachination""",1.0000e-9
"""untranslatable""",7.5858e-8
"""coggly""",1.0000e-9
"""thimblerig""",1.0000e-9
"""Jimmy""",0.000028


In [9]:
df_freq.sort(by='freq', descending=True).write_csv('word_freq.csv', float_precision=9)